In [1]:
import pandas as pd
import torch
import os
import numpy as np
import pickle
from datasets import MultiSetSequence
from torch.utils.data import DataLoader
from sklearn import metrics
import math



### DeepSets Final Predictions


In [2]:

def get_mask(sizes, max_size):
    return (torch.arange(max_size).reshape(1, -1).to(sizes.device) < sizes.unsqueeze(2))


areas = ['north', 'south', 'des']
galaxies = ['lrg', 'elg', 'qso','glbg', 'rlbg']
device = 'cpu'
max_set_len = 0
NSIDE = 512

df_north = pd.read_csv(f'../regression/results/north_complete_{NSIDE}.csv')
df_south = pd.read_csv(f'../regression/results/south_complete_{NSIDE}.csv')
df_des = pd.read_csv(f'../regression/results/des_complete_{NSIDE}.csv')


In [24]:
for area in areas:
    with open(f'data/{area}/{area}_test.pickle', 'rb') as f:
        testset = pickle.load(f)
        f.close()
    if area == "north":
        max_set_len = 30
    elif area == "south":
        max_set_len = 25
    else:
        max_set_len = 40
    df_test = pd.DataFrame.from_dict(testset, orient='index')

    testdata = MultiSetSequence(dict=df_test.to_dict(orient='index'), num_pixels=len(df_test),
                                max_ccds=max_set_len, num_features=6, test=True)

    pixel_id = testdata.pixel_id

    for gal in galaxies:
        testdata.set_targets(gal_type=gal)

        best_val = -100
        for model in os.listdir(f"trained_models/{area}/{gal}"):
            val = float(model[:-3])
            if val > best_val:
                best_val = val

        model = torch.load(f"trained_models/{area}/{gal}/{best_val}.pt",
                           map_location=torch.device('cpu'))

        testloader = torch.utils.data.DataLoader(testdata, batch_size=128, shuffle=False)

        model.eval()
        y_pred = np.array([])
        y_gold = np.array([])

        with torch.no_grad():
            for i, (X1, X2, labels, set_sizes) in enumerate(testloader):
                # Extract inputs and associated labels from dataloader batch
                X1 = X1.to(device)

                X2 = X2.to(device)

                labels = labels.to(device)

                set_sizes = set_sizes.to(device)

                mask = get_mask(set_sizes, X1.shape[2])
                # Predict outputs (forward pass)

                outputs = model(X1, X2, mask=mask)
                # Predict outputs (forward pass)
                # Get predictions and append to label array + count number of correct and total
                y_pred = np.append(y_pred, outputs.cpu().detach().numpy())
                y_gold = np.append(y_gold, labels.cpu().detach().numpy())

            r2 = metrics.r2_score(y_gold, y_pred)
            rmse = math.sqrt(metrics.mean_squared_error(y_gold, y_pred))
            mae = metrics.mean_absolute_error(y_gold, y_pred)

            print()
            print(f" XXXXXX======== TRIAL {area} - {gal} ended")
            print()
            print("Test Set - R-squared: ", r2)
            print("Test Set - RMSE: ", rmse)
            print("Test Set - MAE: ", mae)

        ax = np.stack((pixel_id, y_pred, y_gold), axis=1)
        df_deep = pd.DataFrame(ax, columns=['pixel_id', f'{gal}_deep', 'y_gold'])

        df_deep.pixel_id = df_deep.pixel_id.astype(int)

        if area == 'north':
            df_north = df_north.merge(df_deep, how='inner', on='pixel_id')
        elif area == 'south':
            df_south = df_south.merge(df_deep, how='inner', on='pixel_id')
        else:
            df_des = df_des.merge(df_deep, how='inner', on='pixel_id')


 XXXXXX======== TRIAL north - lrg ended

Test Set - R-squared:  0.019696157369817224
Test Set - RMSE:  11.851315081785096
Test Set - MAE:  8.72075124230638

 XXXXXX======== TRIAL north - elg ended

Test Set - R-squared:  0.2768691500296936
Test Set - RMSE:  19.04504909213889
Test Set - MAE:  14.787118993408097

 XXXXXX======== TRIAL north - qso ended

Test Set - R-squared:  0.1945514680883067
Test Set - RMSE:  26.072144124678687
Test Set - MAE:  20.29797913806312

 XXXXXX======== TRIAL south - lrg ended

Test Set - R-squared:  0.04468225648113022
Test Set - RMSE:  11.550285110350043
Test Set - MAE:  8.420855481319704

 XXXXXX======== TRIAL south - elg ended

Test Set - R-squared:  0.2469150939116309
Test Set - RMSE:  20.181628619139744
Test Set - MAE:  15.66593639433148

 XXXXXX======== TRIAL south - qso ended

Test Set - R-squared:  0.20945053735450814
Test Set - RMSE:  25.0158650292469
Test Set - MAE:  19.2415477733478

 XXXXXX======== TRIAL des - lrg ended

Test Set - R-squared:  0

In [ ]:
print((df_north.head()))
print((df_south.head()))
print((df_des.head()))

In [ ]:
df_north.to_csv(f'../regression/results/north.csv', index=False)
df_south.to_csv(f'../regression/results/south.csv', index=False)
df_des.to_csv(f'../regression/results/des.csv', index=False)

### Final Predictions for the full footprint size



In [2]:

def get_mask(sizes, max_size):
    return (torch.arange(max_size).reshape(1, -1).to(sizes.device) < sizes.unsqueeze(2))


areas = ['north', 'south', 'des']
galaxies = ['lrg', 'elg', 'qso','glbg', 'rlbg']
device = 'cpu'
max_set_len = 0

NSIDE = 512

df_north = pd.read_csv(f'../regression/results/north_complete_{NSIDE}.csv')
df_south = pd.read_csv(f'../regression/results/south_complete_{NSIDE}.csv')
df_des = pd.read_csv(f'../regression/results/des_complete_{NSIDE}.csv')



In [3]:
for area in areas:
    with open(f'data/{area}/{area}_512_robust.pickle', 'rb') as f:
        trainset = pickle.load(f)
        f.close()
    with open(f'data/{area}/{area}_test_512_robust.pickle', 'rb') as f:
        testset = pickle.load(f)
        f.close()

    if area == "north":
        max_set_len = 30
    elif area == "south":
        max_set_len = 25
    else:
        max_set_len = 40
    df_test = pd.DataFrame.from_dict(testset, orient='index')
    df_train = pd.DataFrame.from_dict(trainset, orient='index')
    print(len(df_test), len(df_train))
    df_test = df_test.append(df_train)
    print(len(df_test))



    testdata = MultiSetSequence(dict=df_test.to_dict(orient='index'), num_pixels=len(df_test),
                                max_ccds=max_set_len, num_features=5, test=True)




    pixel_id = testdata.pixel_id


    for gal in galaxies:
        testdata.set_targets(gal_type=gal)

        best_val = -100
        for model in os.listdir(f"trained_models/{area}/{gal}"):
            try:
                int(model[:-3])
                continue

            except:
                val = float(model[:-3])
                if val > best_val:
                    best_val = val

        print(f' Area: {area}. Gal: {gal}. Best val: {best_val}.')
        print()
        model = torch.load(f"trained_models/{area}/{gal}/{best_val}.pt",
                           map_location=torch.device('cpu'))

        testloader = torch.utils.data.DataLoader(testdata, batch_size=128, shuffle=False)

        model.eval()
        y_pred = np.array([])
        y_gold = np.array([])

        with torch.no_grad():
            for i, (X1, X2, labels, set_sizes) in enumerate(testloader):
                # Extract inputs and associated labels from dataloader batch
                X1 = X1.to(device)

                X2 = X2.to(device)

                labels = labels.to(device)

                set_sizes = set_sizes.to(device)

                mask = get_mask(set_sizes, X1.shape[2])
                # Predict outputs (forward pass)

                outputs = model(X1, X2, mask=mask)
                # Predict outputs (forward pass)
                # Get predictions and append to label array + count number of correct and total
                y_pred = np.append(y_pred, outputs.cpu().detach().numpy())
                y_gold = np.append(y_gold, labels.cpu().detach().numpy())

            print(len(y_pred))
            r2 = metrics.r2_score(y_gold, y_pred)
            rmse = math.sqrt(metrics.mean_squared_error(y_gold, y_pred))
            mae = metrics.mean_absolute_error(y_gold, y_pred)

            print()
            print(f" XXXXXX======== TRIAL {area} - {gal} ended")
            print()
            print("Test Set - R-squared: ", r2)
            print("Test Set - RMSE: ", rmse)
            print("Test Set - MAE: ", mae)

        ax = np.stack((pixel_id, y_pred, y_gold), axis=1)
        df_deep = pd.DataFrame(ax, columns=['pixel_id', f'{gal}_deep', 'y_gold'])


        df_deep = df_deep.dropna()
        df_deep.pixel_id = df_deep.pixel_id.astype(int)

        if area == 'north':
            df_north = df_north.merge(df_deep, how='inner', on='pixel_id')
        elif area == 'south':
            df_south = df_south.merge(df_deep, how='inner', on='pixel_id')
        else:
            df_des = df_des.merge(df_deep, how='inner', on='pixel_id')

72616 290460
363076
 Area: north. Gal: lrg. Best val: 0.014333634977770293.



KeyboardInterrupt: 

In [4]:
print((df_north.columns))

print(len(df_north))
print(len(df_south))
print(len(df_des))

Index(['pixel_id', 'lrg', 'elg', 'qso', 'exposures', 'stellar', 'EBV',
       'airmass', 'ccdskysb_g', 'ccdskysb_r', 'ccdskysb_z', 'exptime_g',
       'exptime_r', 'exptime_z', 'meansky_g', 'meansky_r', 'meansky_z',
       'seeing_g', 'seeing_r', 'seeing_z', 'lrg_lin', 'elg_lin', 'qso_lin',
       'lrg_nn', 'elg_nn', 'qso_nn', 'lrg_deep', 'y_gold_x', 'elg_deep',
       'y_gold_y', 'qso_deep', 'y_gold'],
      dtype='object')
97512
185699
86589


In [5]:

print(df_south.head())

   pixel_id  lrg  elg  qso  exposures   stellar       EBV   airmass  \
0    301589   51   97  166   0.453901  0.071429  0.241767  0.327689   
1    357501   35  150  176   0.347518  0.109244  0.116351  0.256850   
2    219584   31  113  138   0.304965  0.260504  0.136105  0.636057   
3    172905   26  107  143   0.588652  0.100840  0.142525  0.485916   
4    192669   22  165  157   0.368794  0.075630  0.074714  0.729906   

   ccdskysb_g  ccdskysb_r  ...     qso_lin     lrg_nn      elg_nn      qso_nn  \
0    0.572745    0.713239  ...  157.051357  33.720192  120.536476  159.514908   
1    0.579899    0.518977  ...  163.857720  34.181618  137.651093  162.061188   
2    0.648851    0.759491  ...  172.237911  34.980652  131.027618  169.156662   
3    0.452184    0.642542  ...  150.417669  33.316101  126.970894  161.554245   
4    0.507952    0.751045  ...  159.760059  32.805458  128.063950  161.738083   

    lrg_deep  y_gold_x    elg_deep  y_gold_y    qso_deep  y_gold  
0  33.693714      5

In [ ]:
to_drop =['pixel_id', 'exposures', 'hinh',
       'gaia', 'gaia12', 'sagitarius', 'stellar', 'EBV', 'airmass',
       'ccdskysb_g', 'ccdskysb_r', 'ccdskysb_z', 'ccdskycounts_g',
       'ccdskycounts_r', 'ccdskycounts_z', 'seeing_g', 'seeing_r', 'seeing_z', 'y_gold_x',
       'y_gold_y', 'y_gold']

df_north = df_north.drop(columns=to_drop, axis=1, inplace=False)
df_south = df_south.drop(columns=to_drop, axis=1, inplace=False)
df_des = df_des.drop(columns=to_drop, axis=1, inplace=False)


In [7]:
df_north.to_csv(f'../regression/results/north_compare.csv', index=False)
df_south.to_csv(f'../regression/results/south_compare.csv', index=False)
df_des.to_csv(f'../regression/results/des_compare.csv', index=False)


